## Pizza Classifier
Determines whether an image contains a pizza<br>
Created for CS50<br>
Dataset: https://www.kaggle.com/datasets/carlosrunner/pizza-not-pizza<br>
Last update: 1/1/24

In [1]:
import cv2
import cupy as np
from importlib import reload
from os import listdir
from sandbox import activations, costs, initializers, layers, model, optimizers, utils

In [2]:
# Load altered pizza dataset
dataset_path = 'dataset\\altered\\'
datasets = [
    np.array([
        cv2.imread(dataset_path + dir + img).flatten()
        for img in listdir(dataset_path + dir)
    ])
    for dir in ['train\\pizza\\', 'train\\not_pizza\\', 'test\\pizza\\', 'test\\not_pizza\\']
]

# Create training and testing data
train_x = np.concatenate((datasets[0], datasets[1]), axis=0) / 255
train_y = np.concatenate((np.ones((datasets[0].shape[0], 1)), np.zeros((datasets[1].shape[0], 1))), axis=0)

test_x = np.concatenate((datasets[2], datasets[3]), axis=0) / 255
test_y = np.concatenate((np.ones((datasets[2].shape[0], 1)), np.zeros((datasets[3].shape[0], 1))), axis=0)

In [33]:
reload(model)

# Configure all modules to run on Cuda
utils.configure_cupy()

# Create model
pizza = model.Model()
pizza.add(layers.Dense(units=32, activation=activations.ReLU()))
pizza.add(layers.Dense(units=16, activation=activations.ReLU()))
pizza.add(layers.Dense(units=8, activation=activations.ReLU()))
pizza.add(layers.Dense(units=1, activation=activations.Sigmoid()))

pizza.configure(
    input_size=train_x.shape[1],
    cost_type=costs.BinaryCrossentropy(),
    optimizer=optimizers.Adam()
)

# Train model
pizza.train(train_x, train_y, epochs=100, batch_size=64, verbose=True)

Cost on epoch 10: 0.5569
Cost on epoch 20: 0.56736
Cost on epoch 30: 0.5481
Cost on epoch 40: 0.38045
Cost on epoch 50: 0.44817
Cost on epoch 60: 0.27721
Cost on epoch 70: 0.40283
Cost on epoch 80: 0.43351
Cost on epoch 90: 0.40456
Cost on epoch 100: 0.35087


In [34]:
# Print model summary
pizza.summary()

# Assess model accuracy
pred_train = pizza.predict(train_x) # Get model accuracy on training data
print('\nTraining Accuracy:', utils.evaluate(pred_train, train_y))
pred_test = pizza.predict(test_x) # Get model accuracy on testing data
print('Testing Accuracy:', utils.evaluate(pred_test, test_y))

+------------+------------+
| Layer type | Parameters |
+------------+------------+
|   Dense    |   393248   |
|   Dense    |    528     |
|   Dense    |    136     |
|   Dense    |     9      |
+------------+------------+
Total parameters: 393921

Training Accuracy: 0.9067880794701987
Testing Accuracy: 0.6370614035087719
